In [4]:
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from bs4 import BeautifulSoup

import locale
from datetime import datetime

In [10]:
def scrap_articles(soup):
    
    article_tags = soup.find_all("article", class_="c-Card c-Card--search")

    articles = [{"title": article.span.text.strip(),
             "abstract": article.find("div", class_="c-Card-content").text.strip(),
             "topic": article.find("span", class_="c-Card-tag").text.strip(),
             "link": article.a["href"],
             "author": article.find("span", class_="c-Card-author").text.strip(),
             "date": convert_timestamp(article.find("span", class_="c-Card-date").text.strip())} for article in article_tags]

    return articles

def convert_timestamp(date_string):

    # Set the locale to French
    locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # For Unix-like systems
    
    # Convert the string to a datetime object
    date_object = datetime.strptime(date_string, '%A %d %B %Y')

    return date_object.isoformat()

def paginate():

    # Wait for the 'Next' button to be clickable
        time.sleep(2)
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.next.page-numbers"))
        )
        # Click the 'Next' button
        next_button.click()
        
        # Optionally, you can add some code here to extract data from each page
        # For example: extract_content_from_page(driver.page_source)

        # Wait for the new page to load
        WebDriverWait(driver, 10).until(
            EC.staleness_of(next_button)
        )

def run_scraping(topic):
    
    scraped_articles = []

    url = f"https://lecourrier.ch/?s={topic}"

    driver.get(url)

    while True:
        try:

            time.sleep(3)
            soup = BeautifulSoup(driver.page_source)
            
            articles = scrap_articles(soup)
            scraped_articles.extend(articles)

            # save articles
            with open(f"../data/scraping/lecourrier_{topic}.json", "w", encoding="utf-8") as fp:
                json.dump(scraped_articles, fp, ensure_ascii=False)
            
            paginate()
                
            
        except Exception as e:
            print("No more pages or error occurred:", e)
            driver.quit()
            break

In [11]:
driver = webdriver.Firefox()

The geckodriver version (0.30.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (130.0.1); currently, geckodriver 0.35.0 is recommended for firefox 130.*, so it is advised to delete the driver in PATH and retry


In [ ]:
topic = "palestine"

run_scraping(topic)